<a href="https://colab.research.google.com/github/mckhannacademy/YugiohCardTextEvaluator/blob/main/yugioh_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

response = requests.get('https://db.ygoprodeck.com/api/v7/cardinfo.php?misc=yes')

In [ ]:
import json

In [ ]:
parsed = json.loads(response.content)
parsed

{'data': [{'id': 34541863,
   'name': '"A" Cell Breeding Device',
   'type': 'Spell Card',
   'frameType': 'spell',
   'desc': 'During each of your Standby Phases, put 1 A-Counter on 1 face-up monster your opponent controls.',
   'race': 'Continuous',
   'archetype': 'Alien',
   'ygoprodeck_url': 'https://ygoprodeck.com/card/a-cell-breeding-device-9766',
   'card_sets': [{'set_name': 'Force of the Breaker',
     'set_code': 'FOTB-EN043',
     'set_rarity': 'Common',
     'set_rarity_code': '(C)',
     'set_price': '0'}],
   'card_images': [{'id': 34541863,
     'image_url': 'https://images.ygoprodeck.com/images/cards/34541863.jpg',
     'image_url_small': 'https://images.ygoprodeck.com/images/cards_small/34541863.jpg',
     'image_url_cropped': 'https://images.ygoprodeck.com/images/cards_cropped/34541863.jpg'}],
   'card_prices': [{'cardmarket_price': '0.08',
     'tcgplayer_price': '0.16',
     'ebay_price': '0.99',
     'amazon_price': '24.45',
     'coolstuffinc_price': '0.25'}],
  

In [ ]:
parsed['data'][5]

{'id': 37478723,
 'name': '"Infernoble Arms - Durendal"',
 'type': 'Spell Card',
 'frameType': 'spell',
 'desc': 'While this card is equipped to a monster: You can add 1 Level 5 or lower FIRE Warrior monster from your Deck to your hand, then destroy this card. If this card is sent to the GY because the equipped monster is sent to the GY: You can target 1 Level 5 or lower FIRE Warrior monster in your GY; Special Summon it, also you cannot Special Summon monsters for the rest of the turn, except Warrior monsters. You can only use 1 ""Infernoble Arms - Durendal"" effect per turn, and only once that turn.',
 'race': 'Equip',
 'archetype': 'Noble Knight',
 'ygoprodeck_url': 'https://ygoprodeck.com/card/infernoble-arms-durendal-10991',
 'card_sets': [{'set_name': '2021 Tin of Ancient Battles',
   'set_code': 'MP21-EN136',
   'set_rarity': 'Super Rare',
   'set_rarity_code': '(SR)',
   'set_price': '0'},
  {'set_name': 'Amazing Defenders',
   'set_code': 'AMDE-EN042',
   'set_rarity': 'Rare',

In [ ]:
name = []
text = []
price = []
for card in parsed['data']:
  name.append(card['name'])
  text.append(card['desc'])
  price.append(float(card['card_prices'][0]['tcgplayer_price']))

In [ ]:
import pandas as pd

In [ ]:
data = {
    "Name": name,
    "Text": text,
    "Price": price
}

In [ ]:
df = pd.DataFrame(data).dropna()

In [ ]:
df[df["Name"].isna()]

,Name,Text,Price
31,None,When your opponent activates a card or effect:...,0.00
51,None,''The king of dinosaurs that rules over the Ea...,0.00
106,None,1 monster with 2500 or more ATK + 1 face-down ...,0.19
177,None,If this card is Special Summoned: You can plac...,0.08
223,None,"When this card is activated: You can add 1 ""Al...",0.00
...,...,...,...
13090,None,"1 ""Yubel"" monster + 1+ Effect Monsters on the ...",6.77
13099,None,If this card is Special Summoned: You can equi...,0.00
13150,None,Special Summon this card as an Effect Monster ...,0.08
13154,None,Activate this Skill during your Main Phase. On...,0.00


In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Split the dataset into training and testing sets
X = df[['Name', 'Text']]
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a column transformer with TfidfVectorizer for "Name" and "Text"
preprocessor = ColumnTransformer(
    transformers=[
        ('name_tfidf', TfidfVectorizer(max_features=25), 'Name'),
        ('text_tfidf', TfidfVectorizer(max_features=50), 'Text')
    ]
)

# Create a pipeline with the preprocessor and a regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Root Mean Squared Error: {np.sqrt(mse)}')


Root Mean Squared Error: 13.033249508972327
